In [1]:
from time import time
import tensorflow as tf
assert tf.__version__ >= "2.0"

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

2021-11-21 00:23:41.203500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-21 00:23:41.203557: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Workflow
<!-- ![flow](../docs/arbre_decision.png) -->

# --------------------------------------------------
# --------- **Welcome to Botes Club Demo** --------
# --------------------------------------------------

## Come have a seat, let me get ready! 

In [2]:
# Load NLP model
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

2021-11-21 00:23:49.335647: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-21 00:23:49.335744: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-21 00:23:49.335788: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-rubrickyard-2dtutorials-2dxqysdus3): /proc/driver/nvidia/version does not exist
2021-11-21 00:23:49.336256: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFCamembertForSequenceCl

### Parser will look for 
- Trigger words to initiate response 
    - Question phrases / punctuations. 
- Sentiment to set "current mood" state
    - Use NLP model
- Start and end time to estimate pauses based on sentence length and set "active listening" state
    - If typed setences has < 10 words per min --> active listening is True

### Exectute response:
 - If trigger word is present
     - If "current mood" is "positive" 
        - If "active listening" state
            - run_compassionate_therapist_bot() --> say "mhmm", "go-on", "I hear you" 
        - Else
            - run_let_me_google_it_bot()
     - Elif "current mood" is "negative"
         - If "active listening" state
            - run_supportive_therapist_bot() --> "Are you not feeling happy?"  
        - Else
            - run_critical_therapist_bot() --> "You need to calm down" 
 - Else:
     - Go back to the beginning 

## Introductions

In [3]:
#@title
import ipywidgets as widgets
from IPython.display import display

class Color:   
   GREEN = '\033[92m'
   RED = '\033[91m'
   BOLD = '\033[1m'   
   END = '\033[0m'

# Widget 1 
button_begin = widgets.Button(
    description='SEND ...',
    button_style='primary'
  )

text_area_begin = widgets.Textarea(
    value='',
    placeholder='What do I call you?',
    description='',
    disabled=False
)

output_begin = widgets.Output()

def on_button_clicked_begin(b):
  text = text_area_begin.value
  # TODO
  # result = simple_text_parser(text)
  result = f"Nice to meet you {text}!"
  print(f'results: {result}')

  with output_begin:    
    print(Color.BOLD + f'{result}: ')

button_begin.on_click(on_button_clicked_begin)
display(text_area_begin, button_begin, output_begin)

Textarea(value='', placeholder='What do I call you?')

Button(button_style='primary', description='SEND ...', style=ButtonStyle())

Output()

## Begin interaction
- Hello there! Tell me what's on your mind? 

In [13]:
# Widget 2
interaction_start_time = time()

def simple_text_parser(txt):
    word_list = txt.split()
    n_words = len(word_list)
    
    # check fof question mark:
    this_is_a_question = False
    last_char = txt[-1]
    if last_char == "?":
        this_is_a_question = True
        
    return n_words, this_is_a_question

button_NLP = widgets.Button(
    description='CLASSIFY !',
    button_style='primary'
  )

text_area_NLP = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='',
    disabled=False
)
output_NLP = widgets.Output()

def on_button_clicked_NLP(b):
  Phrase_end_time = time()
  typing_duration = (Phrase_end_time - interaction_start_time)/60 # in minutes

  text = text_area_NLP.value

  # Simple parser
  n_words, this_is_a_question = simple_text_parser(text)

  type_speed = n_words / typing_duration
  active_listen = False
  if type_speed < 10:
    active_listen = True
  
  # NLP
  result = nlp(text)
  print(f'results: {result}')
  prediction = result[0]["label"]

  if prediction == "POSITIVE":    
    color = Color.GREEN    
  else:
    color = Color.RED

  with output_NLP:    
    print(Color.BOLD + color + f'{prediction}: ' + Color.END + f'"{text}"' + \
          "\n" + f"n_words: {n_words} " + f"Question: {this_is_a_question}" + \
          "\n" + f"active listening: {active_listen} ({typing_duration})")

button_NLP.on_click(on_button_clicked_NLP)
display(text_area_NLP, button_NLP, output_NLP)




Textarea(value='', placeholder='Type something')

Button(button_style='primary', description='CLASSIFY !', style=ButtonStyle())

Output()

## TODO
 - therapist bot functions